# Rendering 3D spherical PLUTO data

# Part 1: generate data
generating the cartesian data out of the original pluto data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import interpn
from astropy import constants as c

Note: I required [this fork of pyPLUTO](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto.git) to work, because of changes in the standard library, see [this fix](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto/-/commit/7b8243c6073785d1486f1bd495a98082f09f95f3)

In [ ]:
import pyPLUTO as pp
au = c.au.cgs.value

In [ ]:
data = pp.pload.pload(10, datatype='flt', w_dir='/Users/birnstiel/Desktop/PLUTO_Planet/data/')

UNIT_DENSITY  = 3.604e-13 # (gr/cm^3)
UNIT_VELOCITY = 6.679e+05 # (cm/s)
UNIT_LENGTH   = 2.992e+14 # (cm)

r = data.x1 * UNIT_LENGTH
ri = data.x1r * UNIT_LENGTH
th = data.x2
thi = data.x2r
ph = data.x3
phi = data.x3r
rho = data.rho.copy() * UNIT_DENSITY

In [ ]:
rhod0 = data.rho * data.tr1 * UNIT_DENSITY

## Inspect the data

Here are the original data dimensions

In [ ]:
r.shape, th.shape, ph.shape

In [ ]:
rho.shape

make a 2D vertical slice

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 151

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rhod0.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rhod0[:, :, i_phi], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{gas}$')

Divide out the mid-plane gradient. This normalizes the disk to mostly take out the general radial gradient.

In [ ]:
rho_mid = rho[:, data.n2//2].mean(-1)
rho_mid_ini = 4e-12 * (data.x1/data.x1[0])**-2.5

In [ ]:
f, ax = plt.subplots()
ax.loglog(data.x1, rho_mid)
ax.loglog(data.x1, rho_mid_ini)

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 151

rho_norm = rhod0 * 100 / rho_mid_ini[:, None, None]

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rho_norm.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rho_norm[:, :, i_phi], norm=LogNorm(vmin=1e-5 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{gas}$')

## Prepare for interpolation

select which density we want to write out, here we take the normalized dust density defined above.

In [ ]:
rho_out = rho_norm

also a bit tricky: we define the density only grid centers, so there is a gap in phi direction between the last and the first grid center as the interpolation will not know about the periodic direction. We close this here by adding another point beyond $2\pi$ which is a copy of the first point near $\phi = 0$.

In [ ]:
ph_mod = np.hstack((ph - ph[0], 2 * np.pi))
rho_mod = np.concatenate((rho_out, rho_out[:, :, 0:1]), axis=2)

We create a cartesian slice, here in the mid-plane `(x, y)` while we call the height `z`

In [ ]:
dx = 0.1

x = np.arange(-40, 40, dx) * au
y = np.arange(-40, 40, dx) * au
z = np.arange(-5, 5, dx) * au

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

We translate the coordinates of that slice to spherical coordinates.

Note that phi goes from 0 to $2 \pi$ in the original data, but the output of `np.arctan2` has negative angles and needs to be shifted

In [ ]:
R = np.sqrt(X**2 + Y**2 + Z**2)
T = np.pi/2 - np.arctan2(Z, np.sqrt(X**2 + Y**2))
P = (np.arctan2(Y, X) + 2 * np.pi) % (2 * np.pi)

We create a array of new points, shape is (N, 3)

In [ ]:
points = np.array([R.ravel(), T.ravel(), P.ravel()]).T

We call the interpolation. There's values outside the box, so we assign them the value `0.0`.

In [ ]:
interp = interpn((r, th, ph_mod), rho_mod, points, fill_value=0.0, bounds_error=False)

The result is again one dimensional (one value per new point), so we need to reshape it to match the shape of the slice. After this, it is again 3-dimensional, since our slice has 3 dimensions, but the z-dimension is just one value here.

In [ ]:
interp = interp.reshape(X.shape)

Plot it

In [ ]:
f, axs = plt.subplots(1, 2, dpi=200, gridspec_kw={'width_ratios':[4, 1]}, figsize=(10, 2))

vmax = 10.**np.ceil(np.log10(interp.max()))

iy = len(y) // 2 
iz = len(z) // 2

ax = axs[0]
cc1 = ax.pcolormesh(X[:, iy, :] / au, Z[:, iy, :] / au, interp[:, iy, :], norm=LogNorm(vmin=1e-3 * vmax, vmax=vmax), shading='auto', rasterized=True)

ax = axs[1]
cc2 = ax.pcolormesh(X[:, :, iz] / au, Y[:, :, iz] / au, interp[:, :, iz], norm=LogNorm(vmin=1e-3 * vmax, vmax=vmax), shading='auto', rasterized=True)


for ax, cc in zip(axs, [cc1, cc2]):
    ax.set_aspect('equal')
    pos = ax.get_position()
    cax = f.add_axes([pos.x0, pos.y1, pos.width, pos.height / 20])
    cb = plt.colorbar(cc, cax=cax, orientation='horizontal')
    cb.set_label('$\\rho_{gas}$')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')

## Export data

store the data in a simple format

In [ ]:
np.savez('../data/pluto_data_norm.npz', x=x, y=y, z=z, rho=interp)

this can be loaded with:

# Part 2: Rendering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from volrender import TransferFunction, Renderer, render_movie
from matplotlib.colors import LogNorm

## Load interpolated data

In [ ]:
with np.load('../data/pluto_data_norm.npz') as f:
    x = f['x']
    y = f['y']
    z = f['z']
    interp = f['rho']

## Rescale data and define the transfer function

In [ ]:
vmax = interp.max()
datacube = LogNorm(vmin=1e-4 * vmax, vmax=vmax, clip=True)(interp.ravel()).reshape(interp.shape)

In [ ]:
tf = TransferFunction(x0=[0.25, 0.6, 0.8], sigma=0.04 * np.ones(3))

tf.x0[:] = [0.25, 0.6, 0.75]
tf.colors[:, -1] = np.array([0.15, 0.05, 0.3])
tf.sigma[:] = np.array([0.05, 0.03, 0.04])

## Render images

Note: this is quite high resolution and takes of the order of a minute to render. Reduce `N` to something like 300 to get results in seconds for testing.

In [ ]:
r = Renderer(datacube, tf=tf, N=800)
r.render(phi=40, theta=60, transparent=False, bg=1.0)

Make two plots, one with white background, the other one transparent.

**Note:** the alpha channel contains the optical depth, i.e. we have to invert it. To get a good looking scaling, we also take it to the fourth power.

In [ ]:
alpha = 1 - r.image[:,:,-1]**4
for i,_alpha in enumerate([1, alpha]):
    f, ax = r.plot(alpha=_alpha)
    ax.set_facecolor('none')
    f.savefig(f'output/disk_{i}.pdf', dpi=600, transparent=True, bbox_inches='tight')

## Render movie

Again: N is quite high, so each frame takes about a minute to render on my laptop. Reduce this to get a (probably $N^3$) speed-up.

In [ ]:
phi = np.linspace(0, 360, 97)[:-1]
fname = 'output/movie_disk.mp4'
render_movie(datacube, theta=60 *np.ones_like(phi), phi=phi, ncpu=1, tf=tf, N=800, dpi=600, bg=1.0, fname=fname)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls loop autoplay>
  <source src="{fname}" type="video/mp4">
</video>
""")